In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import warnings
import awoc
import geopandas as gpd
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.models import Patches
from bokeh.models import ColumnDataSource, Circle
from bokeh.layouts import gridplot
from bokeh.io import show, output_notebook
from bokeh.models import HoverTool

In [2]:
air_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header = None, 
                     names = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone',
                    'DST', 'Tz database time zone', 'Type', 'Source'] )
rout_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat',header = None, 
                     names = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment'])

In [3]:
warnings.filterwarnings('ignore')
#Отключил предупреждения, на работу проги они не влияют

In [4]:
#Задание1
air_raw['IATA'].replace('\\N',np.nan)
air_raw['Country'].replace('\\N',np.nan)
air_raw['Name'].replace('\\N',np.nan)
rout_raw['Source airport'].replace('\\N',np.nan)
rout_raw['Destination airport'].replace('\\N',np.nan)
#посчитаем прилеты
vslet = rout_raw['Source airport'].value_counts()
posadka = rout_raw['Destination airport'].value_counts()
vslet_and_posadka = vslet+posadka
top10 = pd.DataFrame({'Взлеты':vslet,'Прилеты':posadka,'Сумма':vslet_and_posadka})
top10 = top10.fillna(0)
top10['IATA'] = top10.index
air_raw = air_raw.drop_duplicates(subset = 'IATA')
top10['Name'] = top10['IATA'].map(air_raw.set_index('IATA')['Name'])
top10['Country'] = top10['IATA'].map(air_raw.set_index('IATA')['Country'])
top10 = top10.set_index(['Country','Name','IATA'])
arct = air_raw[air_raw['IATA']=='TNM']
top10_1=top10.copy(deep=True)
top10_1.reset_index(inplace= True )
top10 = top10.nlargest(10,'Сумма')

In [5]:
#Добавляем континенты
world= awoc.AWOC()
def get_countinent(name):
    try: cont = world.get_country_continent_name(name)
    except NameError:cont = np.nan
    return cont
air_raw['cont'] = [get_countinent(i) for i in air_raw['Country']]
air_raw[air_raw.cont.isna()]
age = air_raw[air_raw.cont.isna()]['Country'].unique()
age1 = {"Cote d'Ivoire":'Africa', 'Congo (Brazzaville)':'Africa','Congo (Kinshasa)':'Africa','Midway Islands': 'North America', 
       'French Guiana':'South America', 'Martinique':'South America', 'Guadeloupe':'North America','Virgin Islands':'North America',
       'Burma':'Asia', 'Norfolk Island':"Australia", 'Johnston Atoll':"Australia", 'Cocos (Keeling) Islands':"Australia",'Wake Island':'Asia'}
air_raw['cont'] = air_raw['Country'].map(age1).fillna(air_raw['cont'])

In [6]:
africa = gpd.read_file("https://github.com/dodoshkaBEBE/Maps/blob/main/cont_Africa.geojson?raw=true")
asia = gpd.read_file("https://github.com/dodoshkaBEBE/Maps/blob/main/cont_Asia.geojson?raw=true")
europe = gpd.read_file("https://github.com/dodoshkaBEBE/Maps/blob/main/cont_Europe.geojson?raw=true")
north_america = gpd.read_file("https://github.com/dodoshkaBEBE/Maps/blob/main/cont_north_America.geojson?raw=true")
south_america = gpd.read_file("https://github.com/dodoshkaBEBE/Maps/blob/main/cont_south_America.geojson?raw=true")
oceania = gpd.read_file("https://github.com/dodoshkaBEBE/Maps/blob/main/cont_Oceania.geojson?raw=true")
antarctica = gpd.read_file("https://github.com/dodoshkaBEBE/Maps/blob/main/cont_Antarctica.geojson?raw=true")
earth = gpd.read_file('https://github.com/dodoshkaBEBE/Maps/blob/main/earthhh.geojson?raw=true')
#Карты нарезал сам в QGIS, данные взял с https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/

In [7]:
#создаем карты континентов и мира
africa_a1 = GeoJSONDataSource(geojson=africa.to_json())
asia_a1 = GeoJSONDataSource(geojson=asia.to_json())
europe_a1 = GeoJSONDataSource(geojson=europe.to_json())
north_america_a1 = GeoJSONDataSource(geojson=north_america.to_json())
south_america_a1 = GeoJSONDataSource(geojson=south_america.to_json())
oceania_a1 = GeoJSONDataSource(geojson=oceania.to_json())
antarctica_a1 = GeoJSONDataSource(geojson=antarctica.to_json())
earth_a1 = GeoJSONDataSource(geojson=earth.to_json())

africa_fig = figure(title="Africa",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   plot_width=500, 
                   plot_height=300)
asia_fig = figure(title="Asia",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   plot_width=500, 
                   plot_height=300)
europe_fig = figure(title="Europe",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   plot_width=500, 
                   plot_height=300)
north_america_fig = figure(title="North America",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   plot_width=500, 
                   plot_height=300)
south_america_fig = figure(title="South America",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   plot_width=500, 
                   plot_height=300)
oceania_fig = figure(title="Oceania",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   plot_width=500, 
                   plot_height=300)
antarctica_fig = figure(title="Antarctica",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   plot_width=500, 
                   plot_height=300)
earth_fig = figure(title="World Map",
                   x_axis_label="Longitude",
                   y_axis_label="Latitude",
                   x_range=(-180, 180),
                   y_range=(-90, 90),
                   plot_width=500, 
                   plot_height=300)

africa_fig.add_glyph(africa_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))
asia_fig.add_glyph(asia_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))
europe_fig.add_glyph(europe_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))
north_america_fig.add_glyph(north_america_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))
south_america_fig.add_glyph(south_america_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))
oceania_fig.add_glyph(oceania_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))
antarctica_fig.add_glyph(antarctica_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))
earth_fig.add_glyph(earth_a1, Patches(xs="xs", ys="ys", fill_color = 'yellow'))

GlyphRenderer(id='1296', ...)

In [8]:
arct1 = {
    'Country': arct.Country.iloc[0],
    'Name': arct.Name.iloc[0],
    'IATA': arct.IATA.iloc[0],
    'Взлеты': 0,
    'Прилеты': 0,
    'Сумма': 0,
    'Latitude': arct.Latitude.iloc[0],
    'Longitude': arct.Longitude.iloc[0]
}
top10_1 = top10_1.append(arct1, ignore_index=True)
top10_1['cont'] = [get_countinent(i) for i in top10_1['Country']]
top10_1[top10_1.cont.isna()]
top10_1['Latitude'] = top10_1['IATA'].map(air_raw.set_index('IATA')['Latitude'])
top10_1['Longitude'] = top10_1['IATA'].map(air_raw.set_index('IATA')['Longitude'])
top10_1 = top10_1.groupby('cont').apply(lambda x: x.nlargest(5, 'Сумма')).reset_index(drop=True)
continents = ['Asia', 'Africa', 'North America', 'South America', 'Antarctica', 'Europe', 'Oceania']
top10_1_by_continent = {cont: top10_1[top10_1['cont'] == cont] for cont in continents}
top10_1_source_by_continent = {cont: ColumnDataSource(data=dict(lat=top10_1_by_continent[cont]['Latitude'], 
                                                                 lon=top10_1_by_continent[cont]['Longitude'],
                                                                 name=top10_1_by_continent[cont]['Name'])) for cont in continents}
air_raw = air_raw.drop(['Airport ID','City','Country','ICAO','Altitude','Timezone','DST',
                        'Tz database time zone','Type','Source'], axis=1)
airports_by_continent = {cont: air_raw[air_raw['cont'] == cont] for cont in continents}
airports_source_by_continent = {cont: ColumnDataSource(data=dict(lat=airports_by_continent[cont]['Latitude'],
                                                                 lon=airports_by_continent[cont]['Longitude'],name=airports_by_continent[cont]['Name'])) for cont in continents}

In [9]:
#Добавляем аэропорты на карты
africa_fig.add_glyph(airports_source_by_continent['Africa'], Circle(x="lon", y="lat", size=5, fill_color = 'blue'))
asia_fig.add_glyph(airports_source_by_continent['Asia'], Circle(x="lon", y="lat", size=5, fill_color = 'blue'))
europe_fig.add_glyph(airports_source_by_continent['Europe'], Circle(x="lon", y="lat", size=5, fill_color = 'blue'))
north_america_fig.add_glyph(airports_source_by_continent['North America'], Circle(x="lon", y="lat", size=5, fill_color = 'blue'))
south_america_fig.add_glyph(airports_source_by_continent['South America'], Circle(x="lon", y="lat", size=5, fill_color = 'blue'))
oceania_fig.add_glyph(airports_source_by_continent['Oceania'], Circle(x="lon", y="lat", size=5, fill_color = 'blue'))
antarctica_fig.add_glyph(airports_source_by_continent['Antarctica'], Circle(x="lon", y="lat", size=5, fill_color = 'blue'))

africa_fig.add_glyph(top10_1_source_by_continent['Africa'], Circle(x="lon", y="lat", size=12, fill_color = 'red'))
asia_fig.add_glyph(top10_1_source_by_continent['Asia'], Circle(x="lon", y="lat", size=12, fill_color = 'red'))
europe_fig.add_glyph(top10_1_source_by_continent['Europe'], Circle(x="lon", y="lat", size=12, fill_color = 'red'))
north_america_fig.add_glyph(top10_1_source_by_continent['North America'], Circle(x="lon", y="lat", size=12, fill_color = 'red'))
south_america_fig.add_glyph(top10_1_source_by_continent['South America'], Circle(x="lon", y="lat", size=12, fill_color = 'red'))
oceania_fig.add_glyph(top10_1_source_by_continent['Oceania'], Circle(x="lon", y="lat", size=12, fill_color = 'red'))
antarctica_fig.add_glyph(top10_1_source_by_continent['Antarctica'], Circle(x="lon", y="lat", size=12, fill_color = 'red'))
for cont in continents:
    earth_fig.add_glyph(top10_1_source_by_continent[cont], Circle(x="lon", y="lat", size=12, fill_color = 'red'))

output_notebook()

grid = gridplot([
    [africa_fig, asia_fig, europe_fig],
    [north_america_fig, south_america_fig, oceania_fig],
    [antarctica_fig, None, None]
])

 

Loading BokehJS ...

In [10]:
#Добавляем стрелочки на карту, красные - двустороннее авиасообщение, зеленые - односторонее авиасообщение
top_flights = rout_raw[(rout_raw["Source airport"].isin(top10_1["IATA"])) & (rout_raw["Destination airport"].isin(top10_1["IATA"]))]
top_flights['boba'] = top_flights['Source airport'].astype(str) + ', ' +top_flights['Destination airport'].astype(str)
top_flights['boba2'] = top_flights['Source airport'] + top_flights['Destination airport']
top_flights.boba2 = top_flights.boba2.apply(lambda x:''.join(sorted(x)))
pair = top_flights['boba2'].value_counts()
pair = pair.to_frame()
pair['boba2'] = pair['boba2'].apply(lambda x: 'green' if x <= 1 else 'red')
top_flights = top_flights.drop_duplicates(subset = 'boba2')
pair = pair.reset_index().rename(columns={'index': 'New_Index_Name'})
pair = pair.rename(columns={'New_Index_Name': 'boba2', 'boba2': 'Цвет'})
red_df = pd.merge(pair,top_flights, on = 'boba2')
red_df = red_df.drop(["boba2", "Airline",'Airline ID','Source airport ID',
                    'Destination airport ID','Codeshare','Stops','Equipment','boba'], axis=1)
red_df = red_df.merge(air_raw[['IATA', 'Latitude', 'Longitude']], left_on='Source airport', right_on='IATA', how='left').drop(columns='IATA').rename(columns={'Latitude': 'Latitude1', 'Longitude': 'Longitude1'})
red_df = red_df.merge(air_raw[['IATA', 'Latitude', 'Longitude']], left_on='Destination airport', right_on='IATA', how='left').drop(columns='IATA').rename(columns={'Latitude': 'Latitude2', 'Longitude': 'Longitude2'})
green_df = red_df[red_df['Цвет'] == 'green']
red_df = red_df.drop(red_df[red_df['Цвет'] == 'green'].index)
super2 = ColumnDataSource(data=dict(x=[red_df['Longitude1'], red_df['Longitude2']],
                                    y=[red_df['Latitude1'], red_df['Latitude2']]))
earth_fig.multi_line(xs='x', ys='y', source=super2, color = 'blue')

super3 = ColumnDataSource(data=dict(b=[green_df['Longitude1'], green_df['Longitude2']],
                                    d=[green_df['Latitude1'], green_df['Latitude2']]))
earth_fig.line(x='b', y='d', source=super3, color = 'green', line_width=5)


GlyphRenderer(id='1499', ...)

In [11]:
#show(grid)
#show(earth_fig)